### Imports

In [1]:
import sys
from pathlib import Path

# Add the project root directory to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
from config import MODEL_ID, TRUST_REMOTE_CODE, USE_OPENAI_GENERATION
from config import DATA_PATH
from utils.data_preparation_utils import (
    extract_blocks_from_docx, 
    is_valid_block,
    block_to_sample
)
from utils.utils import load_model, load_generator
from utils.prompt_utils import build_prompt
from utils.model_interactions import generate_response, generate_response_openai

from tqdm import tqdm
import json
import pandas as pd

### 📄 Prepare Training Samples from .docx

1. Load the teaching document with dialogs.
2. Extract and filter meaningful blocks.
3. Generate the instruction and system prompt for each block.
4. Use the language model to generate assistant responses.


In [3]:
docx_path = DATA_PATH / "Dialoge neu.docx"
raw_blocks = extract_blocks_from_docx(docx_path)

raw_blocks = [b for b in raw_blocks if is_valid_block(b)]

samples = [block_to_sample(block) for block in raw_blocks]
print("Loaded {} samples".format(len(samples)))

Loaded 27 samples


In [4]:
if not USE_OPENAI_GENERATION:
    tokenizer, model = load_model(MODEL_ID, TRUST_REMOTE_CODE)
    generator = load_generator(model, tokenizer)

### 💾 Save Dataset with Instructions and Responses

Format: one sample per line in `.jsonl`, with fields:
- `system`: system-level instruction
- `instruction`: user question
- `response`: model answer (generated)


#### Open AI  GPT-4

In [5]:
if USE_OPENAI_GENERATION:
    for i, s in enumerate(tqdm(samples, desc="Answer generation (OpenAI)")):
        try:
            gen = generate_response_openai(s["system"], s["instruction"], model="gpt-4")
            s["response"] = gen
        except Exception as e:
            print(f"❌ Error on sample {i}:", e)

Answer generation (OpenAI): 100%|██████████| 27/27 [05:18<00:00, 11.78s/it]


#### Deepseek AI R1-Qwen

In [8]:
if not USE_OPENAI_GENERATION:
    for i, s in enumerate(tqdm(samples, desc="Answer generation")):
        prompt = build_prompt(s["system"], s["instruction"])
        gen = generate_response(generator, prompt, max_tokens=1024)
        response_only = gen[len(prompt):].strip()
        s["response"] = response_only
        print(f"📥 Prompt {i}:", prompt[:120], "...")
        print(f"📤 Response {i}:", response_only[:200], "...")
        print("---")


Answer generation:   4%|▎         | 1/27 [01:20<35:02, 80.86s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📥 Prompt 0: <|im_start|>system
Ты — преподаватель немецкого языка для русскоязычных студентов. Объясняй простыми словами, корректиру ...
📤 Response 0: assierer: Guten Tag! Ich helfe Ihnen gerne weiter.  

2. **Wo ist was?**  
   - Zeichne eine einfache Einrichtung wie ein Bad oder eine Küche und male Möbel und Gegenstände ein.  
   - Frage: Wo ist d ...
---


Answer generation:   7%|▋         | 2/27 [02:31<31:12, 74.89s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📥 Prompt 1: <|im_start|>system
Ты — преподаватель немецкого языка для русскоязычных студентов. Объясняй простыми словами, корректиру ...
📤 Response 1: irm" в singular (нужен один) и "Regenschirme" в plural (нужны несколько) – это правильная форма множественного числа. *Корректно*.

2.  **Du:** "Moment bitte, ich schaue nach." – **Корректно**. Это ст ...
---


Answer generation:  11%|█         | 3/27 [03:42<29:19, 73.30s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

📥 Prompt 24: <|im_start|>system
Ты — преподаватель немецкого языка для русскоязычных студентов. Объясняй простыми словами, корректиру ...
📤 Response 24: овать только предложения уровня A2. Ты должен использовать только предложения уровня A2. Ты должен использовать только предложения уровня A2. Ты должен использовать только предложения уровня A/2. Ты д ...
---


Answer generation:  96%|█████████▋| 26/27 [31:15<01:14, 74.16s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📥 Prompt 25: <|im_start|>system
Ты — преподаватель немецкого языка для русскоязычных студентов. Объясняй простыми словами, корректиру ...
📤 Response 25: "Когда снова..." или "Сколько ещё времени...".
*   **Neue Angebote gibt es jede Woche – ab Montag.** - Ответ тоже правильный. Он использует простую структуру с "gibt es" (есть) и "jede Woche" (каждую  ...
---


Answer generation: 100%|██████████| 27/27 [32:44<00:00, 72.76s/it]

📥 Prompt 26: <|im_start|>system
Ты — преподаватель немецкого языка для русскоязычных студентов. Объясняй простыми словами, корректиру ...
📤 Response 26: annen:** Scan

3.  **Kunde:** "Danke!" (Thank you!)

4.  **Du (Verkäufer):** "Bitte!" (You're welcome!)

**Вопросы и ответы в рамках диалога (A2 Niveau):**

**Frage 1 (Kunde):**  
Was bedeutet "Preiss ...
---


In [6]:
train_path = DATA_PATH / "train_annotated.jsonl"
with open(train_path, "w", encoding="utf-8") as f:
    for s in samples:
        json.dump(s, f, ensure_ascii=False)
        f.write("\n")
print("✅ Saved train_annotated.jsonl")

✅ Saved train_annotated.jsonl


In [7]:
df = pd.DataFrame(samples)
df_review = df[["instruction", "response"]]
df_review.columns = ["User Prompt", "Model Response"]

# Save as Excel and DOCX
xlsx_path = DATA_PATH / "train_annotated_review.xlsx"
df_review.to_excel(xlsx_path, index=False)

import docx
from docx.shared import Pt

doc = docx.Document()
doc.add_heading("AI Responses for Review", 0)

for i, row in df_review.iterrows():
    doc.add_paragraph(f"{i+1}. Prompt:", style='Heading 2')
    doc.add_paragraph(row["User Prompt"])
    doc.add_paragraph("Response:", style='Heading 2')
    doc.add_paragraph(row["Model Response"])
    doc.add_paragraph("\n")

docx_path = DATA_PATH / "train_annotated_review.docx"
doc.save(docx_path)

### 🔁 Convert to Supervised Fine-Tuning (SFT) Format

Each example is structured as:
```json
{
  "messages": [
    {"role": "system", "content": "..."},
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "..."}
  ]
}


In [10]:
with open(train_path, "r", encoding="utf-8") as f:
    raw_samples = [json.loads(line) for line in f if line.strip()]

train_sft_path = DATA_PATH / "train_sft.jsonl"
with open(train_sft_path, "w", encoding="utf-8") as f:
    for s in raw_samples:
        obj = {
            "messages": [
                {"role": "system", "content": s["system"]},
                {"role": "user", "content": s["instruction"]},
                {"role": "assistant", "content": s["response"]}
            ]
        }
        json.dump(obj, f, ensure_ascii=False)
        f.write("\n")

print("✅ Saved train_sft.jsonl")

✅ Saved train_sft.jsonl
